# Pipeline Sloyka class

Импорт модуля Pipeline из Sloyka:

In [ ]:
%pip install sloyka

In [1]:
import sloyka
from sloyka import Pipeline

/Users/sashamorozov/Documents/Code/lib_sloyka/sloyka/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sashamorozov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Передаем модулю неообходимые данные о территории, группе ВК, сервисном ключе и дате, до который выгружать посты:

In [4]:
osm_city_name = 'Санкт-Петербург' # Выбор территории
osm_city_level = 5 # Уровень админ. деления с ОСМ
owner_id = "-129354225" # ID группы ВК ("Центральный район за комфортную городскую среду")
token = '...' # Сервисный ключ доступа из приложения ВК
count = 100 # Кол-во постов в одном запросе
date = "2023-09-20" # Дата, до которой парсятся посты

Запуск модуля Pipeline, на выходе сохраняются и записываются три файла: выгруженные тексты постов, граф в формате `graphml`, визуализированный граф в `html`:

In [ ]:
test = Pipeline(owner_id, token, count, date, osm_city_name, osm_city_level)
posts, graph = test.process()

In [18]:
posts.sample(10)

,date,id,text,views.count,likes.count,reposts.count,type,link,Street,Score
117,2024-01-24 10:30:00,326389,Когда Комитет по благоустройству прекратит год...,3722,59,3,post,NaN,None,NaN
86,2024-02-08 19:30:00,328572,Начало положено! Во вторник вечером состоялась...,4767,89,12,post,https://forms.yandex.ru/u/65a7a4a8c769f1088328...,Конной 24,0.905
43,2024-03-01 10:30:00,331114,Кто-то в Кремле подписан на группу Ц? Вчера пр...,4393,89,6,post,https://vk.com/wall-129354225_160496|восстанав...,Кремле,NaN
16,2024-03-14 19:30:00,332428,"Я художник, я так вижу 🎨 Историческое здание н...",6037,77,8,post,NaN,Полтавской 7,0.976
375,2023-09-20 19:30:00,312204,"Друзья, пока стоит чудесная осенняя погода, вн...",4539,49,7,post,NaN,Овсянниковском саду,NaN
154,2024-01-06 19:30:00,324331,В нашей ежегодной рубрике #отчет2023@mytndvor ...,4357,58,6,post,https://vk.com/wall-129354225_293374|раздельный,Жуковского 43,0.901
356,2023-09-29 19:30:00,312886,"Друзья, а вы бы хотели ещё раз отдохнуть этой ...",3399,32,9,post,NaN,None,NaN
108,2024-01-28 19:30:00,326890,"Вы не забыли, петербуржцы, что во время оттепе...",9284,103,14,post,NaN,Владимирском 3,0.846
331,2023-10-11 19:30:00,314236,Озеленение окончено! Увезли яблоньки с кадок н...,26057,162,16,post,NaN,Восстания,NaN
81,2024-02-11 10:30:00,328892,Вчера состоялись наши зимние активные игры в О...,3539,83,4,post,NaN,Овсянниковском саду,NaN


In [19]:
print(graph)

Graph with 32 nodes and 22 edges


# Pipeline Sloyka step by step

## Установка необходимых для работы библиотек и модулей ##

Установка Sloyka и Pyvis для визуализации семантического графа:

In [2]:
%pip install sloyka
%pip install pyvis

Note: you may need to restart the kernel to use updated packages.


Импорт необходимых модулей:

In [3]:
import sloyka
import warnings

warnings.simplefilter("ignore")

import networkx as nx 
from pyvis.network import Network

## Загрузка данных из ВК ##

Вызов метода для парсинга данных с группы ВК:

In [6]:
from sloyka import VKParser

Ввод `id` группы ВК, с которой будут парситься данные, в `token` сервисный ключ приложения ВК.

In [9]:
owner_id = "-129354225" # группа ВК "Центральный район за комфортную среду обитания", id должен начинаться с "-"
token = '...' # Сервисный ключ доступа из приложения ВК

Запуск парсера, необходимо также уставновить дату, до которой будут выкачиваться тексты постов в формате "гггг.мм.дд.":

In [10]:
posts = VKParser().run_posts(owner_id, token, 100, "2023-09-20") 

2024-01-31 10:30:00
2023-12-13 19:30:00
2023-10-25 10:30:00
2023-09-07 10:30:00
finished


Результат:

In [11]:
posts

,date,id,text,views.count,likes.count,reposts.count,link
0,2024-03-20 10:30:00,333265,Вот и отгуляла широкая Масленица! Наша команда...,1192,27,3,"https://vk.com/wall-129354225_332885|ярко,"
1,2024-03-19 19:30:00,333226,На том же месте сквозь года Вот так каланча ве...,3230,40,11,https://vk.com/wall-129354225_325408|уплотните...
2,2024-03-19 10:30:00,333157,Мы не об этом просили! На углу Восстания и Жук...,6333,72,14,https://vk.com/wall-129354225_315591?access_ke...
3,2024-03-18 19:30:00,333061,Без вашей помощи мы не справимся! Открываем кр...,6574,89,28,https://vk.com/@mytndvor-programma2024|предвыб...
4,2024-03-18 10:30:00,333033,Начинаем новую неделю с хороших новостей: в Пе...,6262,183,9,https://vk.com/wall-129354225_330987|нашей
...,...,...,...,...,...,...,...
395,2023-09-09 10:30:00,311015,Больно терять балконы исторического центра. И ...,5602,98,14,NaN
396,2023-09-08 19:30:00,310989,"Наверняка многие слышали, что ярославские и си...",3363,69,6,NaN
397,2023-09-08 10:30:00,310978,"Сегодня исполняется ровно 82 года, как началас...",7362,144,15,NaN
398,2023-09-07 19:30:00,310962,Осенний кубок Овсянниковского сада по волейбол...,2555,29,6,NaN


Сохранение результата в формате `.csv`:

In [30]:
posts.to_csv('posts.csv', index=False)

## Геокодирование ##

Импорт Геокодера из Sloyka:

In [5]:
from sloyka import Geocoder
geocoder = Geocoder()

2024-03-22 13:54:08,649 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


Запуск гекодера, на вход датафрейм `posts` и колонка `text` из него:

In [14]:
res = geocoder.run(df=posts, text_column='text')

2024-03-20 12:52:41.468 | INFO     | sloyka.src.geocoder:clear_names:256 - clear_names started
2024-03-20 12:52:41.482 | INFO     | sloyka.src.geocoder:get_street:597 - get_street started
2024-03-20 12:52:41.483 | INFO     | sloyka.src.geocoder:get_street:602 - extract_ner_street started
100%|██████████| 400/400 [00:09<00:00, 44.15it/s]
2024-03-20 12:52:58.330 | INFO     | sloyka.src.geocoder:get_street:617 - pattern1.sub started
100%|██████████| 310/310 [00:00<00:00, 335111.92it/s]
2024-03-20 12:52:58.338 | INFO     | sloyka.src.geocoder:get_street:622 - pattern2.findall started
100%|██████████| 310/310 [00:00<00:00, 37611.64it/s]
2024-03-20 12:52:58.352 | INFO     | sloyka.src.geocoder:get_street:630 - pattern2.sub started
100%|██████████| 310/310 [00:00<00:00, 374383.60it/s]
2024-03-20 12:52:58.361 | INFO     | sloyka.src.geocoder:get_street:639 - extract_building_num started
100%|██████████| 310/310 [00:00<00:00, 16593.72it/s]
2024-03-20 12:52:58.389 | INFO     | sloyka.src.geocode

Результат:

In [15]:
res

,date,id,text,views.count,likes.count,reposts.count,link,Street,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,2024-03-19 19:30:00,333226,На том же месте сквозь года Вот так каланча ве...,3230,40,11,https://vk.com/wall-129354225_325408|уплотните...,загородном,NaN,37,Загородном,None,Загородный проспект 37 Санкт-Петербург Россия,['Загородный проспект 37 Санкт-Петербург Россия'],Загородный проспект 37 Санкт-Петербург Россия,Загородный проспект,"Электродом, 37/70 литА, Загородный проспект, о...",POINT (30.33256 59.92240)
1,2024-03-19 10:30:00,333157,Мы не об этом просили! На углу Восстания и Жук...,6333,72,14,https://vk.com/wall-129354225_315591?access_ke...,бадаева,0.971,,Бадаева,None,улица Бадаева Санкт-Петербург Россия,['улица Бадаева Санкт-Петербург Россия'],улица Бадаева Санкт-Петербург Россия,улица Бадаева,"улица Бадаева, округ Правобережный, Санкт-Пете...",POINT (30.45535 59.91689)
2,2024-03-17 10:30:00,332749,Пока по всей стране проводятся выборы Президен...,10181,108,30,NaN,литейном,0.766,24,Литейном,None,"Литейный мост 24 Санкт-Петербург Россия,Литейн...","['Литейный мост 24 Санкт-Петербург Россия', 'Л...",Литейный мост 24 Санкт-Петербург Россия,Литейный мост,"Литейный мост, Разлив, Сестрорецк, Санкт-Петер...",POINT (29.95420 60.08878)
3,2024-03-17 10:30:00,332749,Пока по всей стране проводятся выборы Президен...,10181,108,30,NaN,литейном,0.766,24,Литейном,None,"Литейный мост 24 Санкт-Петербург Россия,Литейн...","['Литейный мост 24 Санкт-Петербург Россия', 'Л...",Литейный мост 24 Санкт-Петербург Россия,Литейный проспект,"Литейный мост, Разлив, Сестрорецк, Санкт-Петер...",POINT (29.95420 60.08878)
4,2024-03-17 10:30:00,332749,Пока по всей стране проводятся выборы Президен...,10181,108,30,NaN,литейном,0.766,24,Литейном,None,"Литейный мост 24 Санкт-Петербург Россия,Литейн...","['Литейный мост 24 Санкт-Петербург Россия', 'Л...",Литейный проспект 24 Санкт-Петербург Россия,Литейный мост,"Дом Мурузи, 24, Литейный проспект, Литейный ок...",POINT (30.34956 59.94247)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,2023-09-10 10:30:00,311065,Вы там нормально себя чувствуете? Решили до ку...,13531,130,13,NaN,адмиралтейской,NaN,,Адмиралтейской,None,Адмиралтейская набережная Санкт-Петербург Рос...,['Адмиралтейская набережная Санкт-Петербург Р...,Адмиралтейская набережная Санкт-Петербург Россия,Адмиралтейская набережная,"Адмиралтейская набережная, Адмиралтейский окру...",POINT (30.30745 59.93892)
394,2023-09-10 10:30:00,311065,Вы там нормально себя чувствуете? Решили до ку...,13531,130,13,NaN,адмиралтейской,NaN,,Адмиралтейской,None,Адмиралтейская набережная Санкт-Петербург Рос...,['Адмиралтейская набережная Санкт-Петербург Р...,Адмиралтейская набережная Санкт-Петербург Россия,Адмиралтейская улица,"Адмиралтейская набережная, Адмиралтейский окру...",POINT (30.30745 59.93892)
395,2023-09-10 10:30:00,311065,Вы там нормально себя чувствуете? Решили до ку...,13531,130,13,NaN,адмиралтейской,NaN,,Адмиралтейской,None,Адмиралтейская набережная Санкт-Петербург Рос...,['Адмиралтейская набережная Санкт-Петербург Р...,Адмиралтейская улица Санкт-Петербург Россия,Адмиралтейская набережная,"Адмиралтейская улица, Близнецы, Колпино, Санкт...",POINT (30.59574 59.74681)
396,2023-09-10 10:30:00,311065,Вы там нормально себя чувствуете? Решили до ку...,13531,130,13,NaN,адмиралтейской,NaN,,Адмиралтейской,None,Адмиралтейская набережная Санкт-Петербург Рос...,['Адмиралтейская набережная Санкт-Петербург Р...,Адмиралтейская улица Санкт-Петербург Россия,Адмиралтейская улица,"Адмиралтейская улица, Близнецы, Колпино, Санкт...",POINT (30.59574 59.74681)


Сохранение результата в формате `.csv`:

In [29]:
res.to_csv('data_posts.csv', index=False) 

## Построение семантического графа ##

Импорт метода для построения семантического графа:

In [6]:
from sloyka import Semgraph
sm = Semgraph(device='cpu')

Сборка семантического графа на основе полученных данных:

In [37]:
# На вход df с геокодированными данными
G = sm.build_semantic_graph(res,
                        'text',
                        'only_full_street_name', 
                        'initial_street',
                        'Toponims',
                        key_score_filter=0.6,       # скор для извлечения слов из сообщений. Больше - меньше слов в графе, меньш- больше слов в графе (хуже смысл)
                        semantic_score_filter=0.75) # скор для построения связей между извлечёнными словами. Больше - меньше связей в графе, меньш - больше связей в графе (хуже смысл)

Extracting keywords


100%|██████████| 62/62 [00:34<00:00,  1.79it/s]


Keywords extracted
Calculating semantic closeness


100%|██████████| 276/276 [00:03<00:00, 79.93it/s]


Сохранение графа в формате `graphml`:

In [39]:
nx.write_graphml(G, 'sem_graph.graphml', encoding='utf-8')

Визуализация графа в формате `html`:

In [42]:
nt = Network('1000px', '1000px')
nt.show_buttons(filter_=['physics'])
nt.from_nx(G)

nt.write_html('graph.html')